In [14]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [15]:
dataset = pd.read_csv('Churn_Modelling.csv')
# Assuming that the first three columns do not affect outcome (rownumber, custId, surname)
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [16]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [17]:
print(y)

[1 0 1 ... 1 1 0]


In [18]:
# NOTE THAT THERE IS NOT DATA MISSING

# Encoding categorical data (Country origin (ONE HOT) and Gender (1 or 0)) 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [19]:
# One hot encoding country 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [20]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [7]:
# Spitting dataset into training and test set
from sklearn.model_selection import train_test_split
# Random_State is random seed
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# For deeplearning we must scale all the features
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
# Building the ANN
# Initializing it
ann = tf.keras.models.Sequential()

# Adding input layer, first and second hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

# Adding output layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [10]:
# Training ANN (adam uses stochastic descent)
ann.compile(optimizer='adam', loss= 'binary_crossentropy', metrics= ['accuracy'])
ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5468 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4738 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4455 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4351 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4299 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4267 - accuracy: 0.8011
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4237 - accuracy: 0.8198
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4207 - accuracy: 0.8236
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4180 - accuracy: 0.8241
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4152 - accura

In [22]:
# Predicting probability for a Male(1) with Credit Score of 600, from France (1,0,0), 40 year old, tenure of 3 years
# balance of 600000, 2 products bought, has a credit card(1), active(1) and a salary of 50000
# REMEBER: Hot one encoded vars go first, data needs to be scaled to our model as well
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])))

[[0.05505573]]


In [23]:
# Final prediction (threshold of 50%)
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

[[False]]


In [26]:
# Use test data set to predict results (get final prediction, not probability)
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [27]:
# Making the confusion matrix
# TP FP
# FN TN
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1489  106]
 [ 178  227]]


0.858